# <span style="color:#a4d4a3">**Feature-based SLAM with Particle Filters (FastSLAM 1.0)**</span>

Within this module we <span style="color:#ffa500">**formulate and implement**</span> a feature-based SLAM using a <span style="color:#ffa500">**Rao–Blackwellized Particle Filter**</span> (FastSLAM).

### <span style="color:#a4d4a3">**Particle Representation**</span>

As we saw in the previous module, the particles are a <span style="color:#00703c">**set of weighted samples**</span>:

$$
\mathcal{X} = \left\{\left\langle x^{[i]},\,w^{[i]}\right\rangle\right\}_{i=1,\ldots,N}
$$

where each sample is a <span style="color:#ffa500">**hypothesis about the state**</span>.

As we saw in the Kalman Filter modules, for feature–based SLAM, the state is:

$$
x=(x_{1:t}, m_{1,x}, m_{1,y}, m_{2,x}, m_{2,y}, \ldots, m_{M,x}, m_{M,y})^T
$$

where $x_{1:t}$ are robot <span style="color:#ffa500">**poses**</span> and $m_j$ are the <span style="color:#ffa500">**landmark positions**</span>.

##### 📏 <span style="color:#a4d4a3">Dimensionality Problem</span>

The Particle Filters are effective in <span style="color:#ffa500">**low-dimensional**</span> spaces as the likely regions of the state space need to be covered with samples. The above state space is <span style="color:#ffa500">**high-dimensional!**</span>

*Can we exploit <span style="color:#00703c">**dependencies between the different dimensions**</span> of the state space?*

- *If we* <span style="color:#00703c">***know the poses***</span> *of the robot,* <span style="color:#00703c">***mapping is easy!***</span>

<span style="color:#ffa500">**Key Idea:**</span>

- If we use the particle set only to model the robot's path, <span style="color:#ffa500">**each sample is a path hypothesis**</sample>. 

- For each sample, we can compute <span style="color:#ffa500">**an individual map of landmarks**</span>.

##### ⛏️ <span style="color:#a4d4a3">Rao-Blackwellization</span>

Rao-Blackwellization is a trivial <span style="color:#ffa500">**factorization**</span> that exploits dependencies between variables:

$$
p(a, b) = p(b \mid a) \cdot p(a)
$$

If $p(b \mid a)$ can be <span style="color:#ffa500">**computed efficiently**</span>, then represent only $p(a)$ with samples and compute $p(b \mid a)$ for every sample.

<span style="color:#00703c">**In our case:**</span>

- $a$ is the trajectory of the robot
- $b$ is the map

Thus we can say, <span style="color:#ffa500">**instead of computing**</span> $p(a,b)$, let's <span style="color:#ffa500">**take only**</span> $p(a)$, <span style="color:#ffa500">**represent it with samples**</span>, then compute $p(b \mid a)$, <span style="color:#ffa500">**which is the map given the trajectory**</span>. 

Because we said that for grid maps, <span style="color:#ffa500">**mapping is easy if we know the poses!**</span>

---

### 🛠️ <span style="color:#a4d4a3">**Rao-Blackwellization for SLAM**</span>

Let's apply this factorization to the SLAM posterior:

$$
p(x_{0:t}, m_{1:M} \mid z_{1:t},u_{1:t}) = p(x_{0:t} \mid z_{1:t}, u_{1:t}) \cdot p(m_{1:M} \mid x_{0:t}, z_{1:t}) \quad\quad (1)
$$

where:

- $p(x_{0:t} \mid z_{1:t}, u_{1:t})$ is the <span style="color:#ffa500">**path posterior**</span>.
- $p(m_{1:M} \mid x_{0:t}, z_{1:t})$ is the <span style="color:#ffa500">**map posterior**</span>.

> 📝 <span style="color:#0098ff">**Note:**</span> <em>This was first introduced for SLAM by Murphy in 1999. </em>

*Now, how do we* <span style="color:#ffa500">***compute the map posterior***</span> *efficiently?*

Because we factor the problem into the robot path and the map, and compute the map given the poses, the <span style="color:#ffa500">**landmarks are conditionally independent given the poses**</span>; hence, from Eq. (1) we can write:

$$
p(x_{0:t}, m_{1:M} \mid z_{1:t},u_{1:t}) = p(x_{0:t} \mid z_{1:t}, u_{1:t}) \cdot \prod_{i=1}^{M} p(m_i \mid x_{0:t}, z_{1:t})
$$

With this factorization, we use:

- <span style="color:#ffa500">**MCL**</span> to estimate the <span style="color:#ffa500">**robot poses**</span> 
- <span style="color:#ffa500">**2-D EKFs**</span> (one per landmark) to maintain <span style="color:#ffa500">**the map**</span>. 

Managing M independent 2×2 EKFs is more efficient than a single large joint filter over all landmarks.

> 📝 <span style="color:#0098ff">**Note:**</span> <em>This was first exploited in FastSLAM by Montemerlo et al., in 2002. </em>

### 🛣️ <span style="color:#a4d4a3">**Modeling the Robot's Path**</span>

We use the <span style="color:#ffa500">**sample-based representation**</span> for $p(x_{0:t} \mid z_{1:t}, u_{1:t})$

Each sample is a path hypothesis:

- $x_0$ starting location, typically $(0,0,0)$
- $x_1$ pose hypothesis at time $t=1$
- $x_2$ pose hypothesis at time $t=2$
- $\ldots$

Since the Particle Filter <span style="color:#ffa500">**does not update past poses**</span> if a mistake was made since they are not revised, we <span style="color:#ffa500">**do not need to maintain them**</span> in the sample set.

---

### ⚡️ <span style="color:#a4d4a3">**FastSLAM**</span>

This leads to the <span style="color:#ffa500">**first efficient SLAM implementation**</span>, known as FastSLAM by *Montemerlo et al.* in 2002.

- Each <span style="color:#ffa500">**landmark**</span> is represented by a <span style="color:#ffa500">**2×2 EKF**</span>.
- Each particle maintains <span style="color:#ffa500">**$M$ independent**</span> EKFs for its landmarks.

We have two updates:
  - <span style="color:#00703c">**Action update:**</span> sample a new pose using the proposal distribution (motion model).
  - <span style="color:#00703c">**Sensor update:**</span> EKF on observed landmarks.

##### 🔑 <span style="color:#a4d4a3"> **Key Steps of FastSLAM 1.0** </span>

1. The first step is to <span style="color:#ffa500">**extend**</span> the path posterior by <span style="color:#ffa500">**sampling**</span> a new pose for each sample
    
    $$
    x_t^{[k]} \sim p(x_t \mid x_{t-1}^{[k]}, u_t)
    $$

2. Then we the <span style="color:#ffa500">**compute particle weights**</span> via the observation likelihood
   
   $$
   w^{[k]}= |2 \pi Q|^{-1/2} \exp{\Big(-\tfrac{1}{2}\,(z_t- \hat{z}^{[k]})^T Q^{-1} (z_t - \hat{z}^{[k]})\Big)}
   $$

3. <span style="color:#ffa500">**Update**</span> the belief of the <span style="color:#ffa500">**observed**</span> landmarks using the EKF(s) update rule.

4. <span style="color:#ffa500">**Resample**</span> particles.

#### 👨🏻‍💻 <span style="color:#a4d4a3">**FastSLAM 1.0 Algorithm**</span>

The complete <span style="color:#ffa500">****FastSLAM 1.0**</span> algorithm can be summarized as follows:

> <tt> <span style="color:#4D96FF">def</span> **<span style="color:#6BCB77">FastSLAM1.0</span>($\color{#ffa500}\mathcal{X}_{t-1}, \color{#ffa500}c_t, \color{#ffa500}u_t, \color{#ffa500}z_t$):** 
>>
>> <span style="color:#FF2DD1">1.</span> <span style="color:#e74c3c">for</span> $k=1$ <span style="color:#ffa500">to</span> $N$ <span style="color:#e74c3c">do</span>: $\quad$ <span style="color:#948979"># Loop over all particles</span>
>>>
>>> <span style="color:#FF2DD1">2.</span> <span style="color:#e74c3c">Let</span> $\big\langle x_{t-1}^{[k]}, \langle \mu_{1,t-1}^{[k]}, \Sigma_{1,t-1}^{[k]}\rangle, \ldots \big\rangle$ <span style="color:#ffa500">be particle</span> $k$ <span style="color:#e74c3c">in</span> $\mathcal{X}_{t-1}$
>>> 
>>> <span style="color:#FF2DD1">3.</span> $x_{t}^{[k]} \sim p(x_t \mid x_{t-1}^{[k]}, u_t) \quad$ <span style="color:#948979"># Sample pose</span>
>>>
>>> <span style="color:#FF2DD1">4.</span> $j = c_t \quad $ <span style="color:#948979"># Observed feature</span>
>>>
>>> <span style="color:#FF2DD1">5.</span> <span style="color:#e74c3c">if</span> <span style="color:#ffa500"> feature</span> $j$ <span style="color:#ffa500">Never seen before</span>
>>>>
>>>> <span style="color:#FF2DD1">6.</span> $\mu_{j,t}^{[k]} = h^{-1}(z_t, x_t^{[k]})\quad $ <span style="color:#948979"># Initialize mean</span>
>>>> 
>>>> <span style="color:#FF2DD1">7.</span> $H = h'(\mu_{j,t}^{[k]}, x_t^{[k]}) \quad$ <span style="color:#948979"># Calculate Jacobian</span>
>>>>
>>>> <span style="color:#FF2DD1">8.</span> $\Sigma_{j,t}^{[k]} = H^{-1}\,Q_t\,(H^{-1})^T \quad$ <span style="color:#948979"># Initialize covariance</span>
>>>>
>>>> <span style="color:#FF2DD1">9.</span> $w^{[k]} = p_0 \quad$ <span style="color:#948979"># Default importance weight</span>
>>>
>>> <span style="color:#FF2DD1">10.</span> <span style="color:#e74c3c">else</span>
>>>>
>>>> <span style="color:#FF2DD1">11.</span> $\hat{z}^{[k]} = h(\mu_{j,t-1}^{[k]}, x_t^{[k]}) \quad$ <span style="color:#948979"># Measurement Prediction</span>
>>>>
>>>> <span style="color:#FF2DD1">12.</span> $H = h'(\mu_{j,t-1}^{[k]}, x_t^{[k]}) \quad$ <span style="color:#948979"># Calculate Jacobian</span>
>>>>
>>>> <span style="color:#FF2DD1">13.</span> $Q = H\,\Sigma_{j,t-1}^{[k]}\,H^T + Q_t \quad$ <span style="color:#948979"># Compute measurement covariance</span>
>>>>
>>>> <span style="color:#FF2DD1">14.</span> $K = \Sigma_{j,t-1}^{[k]}\,H^T \, Q^{-1} \quad$ <span style="color:#948979"># Calculate Kalman gain</span>
>>>>
>>>> <span style="color:#FF2DD1">15.</span> $\mu_{j,t}^{[k]} = \mu_{j,t-1}^{[k]} + K\,(z_t - \hat{z}^{[k]})^T \quad$ <span style="color:#948979"># Update mean</span>
>>>>
>>>> <span style="color:#FF2DD1">16.</span> $\Sigma_{j,t}^{[k]} = (I - K\,H)\,\Sigma_{j,t-1}^{[k]} \quad$ <span style="color:#948979"># Update covariance</span>
>>>> 
>>>> <span style="color:#FF2DD1">17.</span> $w^{[k]} = | 2 \pi Q |^{-1/2} \exp{\Big(-\tfrac{1}{2} (z_t - \hat{z}^{[k]})^T Q^{-1} (z_t - \hat{z}^{[k]})\Big)} \quad$ <span style="color:#948979"># Compute importance facotr</span>
>>>> 
>>> <span style="color:#FF2DD1">18.</span> <span style="color:#e74c3c">endif</span>
>>>
>>> <span style="color:#FF2DD1">19.</span> <span style="color:#e74c3c">for</span> <span style="color:#ffa500"> all unobserved features</span> $j'$ <span style="color:#e74c3c">do</span>
>>>>
>>>> <span style="color:#FF2DD1">20.</span> $\langle \mu_{j',t}^{[k]}, \Sigma_{j',t}^{[k]} \rangle = \langle \mu_{j',t-1}^{[k]}, \Sigma_{j',t-1}^{[k]} \rangle \quad$ <span style="color:#948979"># Leave unchanged</span>
>>>
>>> <span style="color:#FF2DD1">21.</span> <span style="color:#e74c3c">endfor</span>
>>
>> <span style="color:#FF2DD1">22.</span> <span style="color:#e74c3c">endfor</span>
>>
>> <span style="color:#FF2DD1">23.</span> $\mathcal{X}_t = $ <span style="color:#6BCB77">Resample</span>($\big\langle x_t^{[k]}, \langle \mu_{1,t}^{[k]}, \Sigma_{1,t}^{[k]} \rangle,\ldots, w^{[k]} \big\rangle_{k=1..N}$)  
>>
>> <span style="color:#FF2DD1">24.</span> <span style="color:#e74c3c">return</span> $\mathcal{X}_t$

---

### ⚖️ <span style="color:#a4d4a3">**The Importance Weight**</span>

As we mentioned earlier <span style="color:#ffa500">**the weight is a result from the Importance Sampling Principle**</span>.

- Importance weight is given by the ratio of target and proposal in $x^{[k]}$

    $$
    w^{[k]} = \frac{\text{target}({x^{[k]}})}{\text{proposal}(x^{[k]})} \quad\quad (2)
    $$

Let's see the mathematical derivation of it.

- The <span style="color:#ffa500">**target distribution**</span> is the path of the robot:

    $$
    p(x_{1:t} \mid z_{1:t}, u_{1:t})
    $$

- The <span style="color:#ffa500">**proposal distribution**</span> is all the odometry information, except the last observation:

    $$
    p(x_{1:t} \mid z_{1:t-1}, u_{1:t})
    $$

- We can write the proposal in a <span style="color:#ffa500">**recursive**</span> step-by-step:

    $$
    p(x_{1:t} \mid z_{1:t-1}, u_{1:t}) = \underbrace{p(x_{t} \mid x_{t-1}, u_{t})}_{\text{from } \mathcal{X}_{t-1} \text{ to } \bar{\mathcal{X}}_t} \cdot \underbrace{p(x_{1:t-1} \mid z_{1:t-1}, u_{1:t-1})}_{\mathcal{X}_{t-1}}
    $$

Replacing the target and proposal distributions in Eq. (2) we get:

$$
\begin{aligned}
w^{[k]} &= \frac{\text{target}({x^{[k]}})}{\text{proposal}(x^{[k]})} \\
&= \frac{p(x_{1:t}^{[k]} \mid z_{1:t}, u_{1:t})}{p(x_{t}^{[k]} \mid x_{t-1}^{[k]}, u_{t}) \cdot p(x_{1:t-1}^{[k]} \mid z_{1:t-1}, u_{1:t-1})} \quad\quad (3) \\
\end{aligned}
$$

Now if we apply <span style="color:#ffa500">**Bayes' rule**</span> and <span style="color:#ffa500">**factorization**</span> to Eq. (3), we get:

$$
\begin{aligned}
w^{[k]} &= \frac{\text{target}({x^{[k]}})}{\text{proposal}(x^{[k]})} \\
&= \frac{p(x_{1:t}^{[k]} \mid z_{1:t}, u_{1:t})}{p(x_{t}^{[k]} \mid x_{t-1}^{[k]}, u_{t}) \cdot p(x_{1:t-1}^{[k]} \mid z_{1:t-1}, u_{1:t-1})} \\
&= \frac{\eta \cdot p(z_{t} \mid x_{1:t}^{[k]}, z_{1:t-1}) \cdot \cancel{p(x_{t}^{[k]} \mid x_{t-1}^{[k]}, u_t)} \cdot \cancel{p(x_{1:t-1}^{[k]} \mid z_{1:t-1}, u_{1:t-1})}}{\cancel{p(x_{t}^{[k]} \mid x_{t-1}^{[k]}, u_{t})} \cdot \cancel{p(x_{1:t-1}^{[k]} \mid z_{1:t-1}, u_{1:t-1})}} \\
&= \eta \cdot p(z_{t} \mid x_{1:t}^{[k]}, z_{1:t-1}) \quad\quad (4) \\
\end{aligned}
$$

By integrating over the observed landmark position, we get:

$$
\begin{aligned}
w^{[k]} &= \eta \cdot p(z_{t} \mid x_{1:t}^{[k]}, z_{1:t-1}) \\
&= \eta \int p(z_{t} \mid x_{1:t}^{[k]}, z_{1:t-1}, m_j) \cdot p(m_j \mid x_{1:t}^{[k]}, z_{1:t-1}) \cdot dm_j \\
&= \eta \int \underbrace{p(z_{t} \mid x_{t}^{[k]}, m_j)}_{\mathcal{N}(z_t;\hat{z}^{[k]},Q_t)} \cdot \underbrace{p(m_j \mid x_{1:t}^{[k]}, z_{1:t-1})}_{\mathcal{N}(m_j;\mu_{j,t-1}^{[k]}, \Sigma_{j,t-1}^{[k]})} \cdot dm_j \\  
\end{aligned}
$$

Here we have:

- $p(m_j \mid x_{1:t}^{[k]}, z_{1:t-1})$: the <span style="color:#ffa500">**landmark prior**</span> in particle $k$, a Gaussian $\mathcal{N}\!\big(m_j;\, \mu_{j,t-1}^{[k]}, \Sigma_{j,t-1}^{[k]}\big)$.

- $p(z_t \mid x_t^{[k]}, m_j)$: the <span style="color:#ffa500">**measurement model**</span> with sensor noise $Q_t$, $\,\mathcal{N}\!\big(z_t;\, h(x_t^{[k]}, m_j),\, Q_t\big)$.

- $\hat z^{[k]} \approx h\!\big(x_t^{[k]}, \mu_{j,t-1}^{[k]}\big)$: the <span style="color:#ffa500">**predicted measurement**</span> (using a first-order linearization w.r.t. the landmark).

The total measurement covariance is the <span style="color:#ffa500">**projected landmark uncertainty**</span> plus the <span style="color:#ffa500">**sensor noise**</span>:

$$
Q \;=\; H\,\Sigma_{j,t-1}^{[k]} H^\top \;+\; Q_t .
$$

where $H$ is the <span style="color:#ffa500">**Jacobian**</span> of the measurement model w.r.t. the landmark. 

The <span style="color:#ffa500">**marginal likelihood**</span> after integrating out $m_j$ is Gaussian, so

$$
w^{[k]} \;\approx\; |2\pi Q|^{-1/2}\,
\exp\!\Big(-\tfrac12\,(z_t-\hat z^{[k]})^\top Q^{-1}(z_t-\hat z^{[k]})\Big).
$$

> 📝 <span style="color:#0098ff">**Note:**</span> <em>The weight tells us how consistent is the world representation each sample generated with respect to what the system actually perceives. </em>

---

##### 🎮 `Python Example: FastSLAM - Feature-based SLAM`

Since the desired algorithm has <span style="color:#ffa500">**grown in complexity**</span>, we break it into the following three blocks.

1. **FastSLAM core**
2. **Map and LiDAR handling**
3. **Pygame demo app**

##### ⚙ <span style="color:#a4d4a3">**FastSLAM Core** (RBPF with per-landmark EKFs)</span>

We start with this cell which implements a solution similar to what <span style="color:#00703c">**FastSLAM 1.0**</span> proposed:

- A Rao-Blackwellized Particle Filter over the <span style="color:#ffa500">**robot pose**</span> where each particle carries an <span style="color:#ffa500">**independent landmark map**</span> represented by tiny 2×2 EKFs (position only).

<span style="color:#a4d4a3">**Assumptions / scope:**</span>

- <span style="color:#00703c">**Known data association:**</span> the front-end provides an integer landmark ID for each measurement; the core does *not* solve association.
- <span style="color:#00703c">**2D state:** </span>robot pose $(x,y,\theta)$; each landmark $m_j \in \mathbb{R}^2$.
- <span style="color:#00703c">**Measurements:**</span> range-bearing $z = (r,\phi)$ w.r.t. the robot frame with Gaussian noise $Q\in\mathbb{R}^{2\times2}$.
- <span style="color:#00703c">**Motion model:**</span> odometry $(\Delta r_1,\Delta t,\Delta r_2)$ with Gaussian noise $R_x$.

<span style="color:#a4d4a3">**Per-time-step flow:**</span>

1. <span style="color:#00703c">**Predict (particles):**</span> sample each particle pose from the odometry proposal.
2. <span style="color:#00703c">**Correct (EKFs inside each particle):**</span>
   - Measurement model $h(x,m)$.
   - Jacobian $H = \partial h/\partial m \in \mathbb{R}^{2\times2}$.
   - For a new landmark ID, initialize $\mu_j$ via inverse sensor model and $\Sigma_j$ from a local linearization.
   - For an existing landmark, do an EKF update. We compute the Kalman gain via a numerically stable linear solve rather than explicit matrix inversion ($K = \Sigma H^T (H \Sigma )$).
   - Accumulate the measurement log-likelihood to update the particle weight.
3.  <span style="color:#00703c">**Normalize weights**</span> and compute the effective sample size $N_\mathrm{eff}$.
4.  <span style="color:#00703c">**Systematic resampling**</span> when $N_\mathrm{eff} < \text{ratio}\times N$.

In [6]:
# ====================================================
# FastSLAM 1.0 Pygame Demo (GT left | FastSLAM right)
# ====================================================

import numpy as np

# --- Small helpers ---
def wrap_angle(a: float) -> float:
    return (a + np.pi) % (2*np.pi) - np.pi

# --- FastSLAM core ---
class FastSLAM:
    class Particle:
        __slots__ = ("x", "y", "th", "w", "map") 
        def __init__(self, x=0.0, y=0.0, th=0.0, w=1.0):
            self.x  = float(x)
            self.y  = float(y)
            self.th = float(th)
            self.w  = float(w)
            # map: lm_id -> [mu(2,), Sigma(2,2), seen_bool]
            self.map = {}

    # --- Constructor ---
    def __init__(self,
                 N: int,
                 Q: np.ndarray,
                 R_x: np.ndarray,
                 rng: np.random.Generator,
                 resample_neff_ratio: float = 0.5):
        """
        FastSLAM 1.0 with known data association and per-landmark EKFs.
        Args:
            N: number of particles
            Q: measurement noise covariance (2x2)
            R_x: odometry noise covariance (3x3, diagonal)
            rng: numpy random number generator
            resample_neff_ratio: trigger resampling when N_eff < ratio * N
        """
        self.N = int(N)
        self.Q = np.asarray(Q, dtype=np.float32)
        self.R_x = R_x
        self.std = np.sqrt(np.diag(R_x))
        self.rng = rng
        self.resample_neff_ratio = float(resample_neff_ratio)
        self.p = [self.Particle() for _ in range(self.N)]


    # --- Initialization ---
    def global_initialize(self, cx: float, cy: float,
                          th_sd_deg: float = 10.0, pos_sd: float = 3.0) -> None:
        """
        Spread particles around (cx,cy) with small heading noise.
        Args:
            cx, cy: center position
            th_sd_deg: heading stddev in degrees
            pos_sd: position stddev in world units
        """
        th_sd = np.deg2rad(th_sd_deg)
        invN = 1.0 / self.N
        # vectorized sampling of random offsets
        xs = cx + self.rng.normal(0.0, pos_sd, size=self.N)
        ys = cy + self.rng.normal(0.0, pos_sd, size=self.N)
        ths = self.rng.normal(0.0, th_sd, size=self.N)
        for i, pr in enumerate(self.p):
            pr.x  = float(xs[i])
            pr.y  = float(ys[i])
            pr.th = float(ths[i])
            pr.w  = invN
            pr.map.clear()

    # --- Motion / Prediction ---
    def predict(self, u: np.ndarray) -> None:
        """
        Odometry motion with additive diagonal Gaussian noise.
        Args:
            u: odometry 3-vector (dr1, dt, dr2)
        Modifies:
            self.p: list of particles (in-place)
        """
        dr1, dt, dr2 = map(float, u)
        n = self.N

        x  = np.empty(n, np.float64)
        y  = np.empty(n, np.float64)
        th = np.empty(n, np.float64)
        for i, pr in enumerate(self.p):
            x[i] = pr.x; y[i] = pr.y; th[i] = pr.th

        # Odometry motion (noiseless)
        th_mid = th + dr1
        ct = np.cos(th_mid); st = np.sin(th_mid)
        x_new  = x  + dt * ct
        y_new  = y  + dt * st
        th_new = th + dr1 + dr2

        # Add independent Gaussian noise per component
        std = self.std  # [σx, σy, σθ]
        eps = self.rng.normal(0.0, std, size=(n,3))

        # Rotate translational noise from body to world
        ex_b = eps[:, 0]
        ey_b = eps[:, 1]
        eth  = eps[:, 2]

        ex_w = ct * ex_b - st * ey_b
        ey_w = st * ex_b + ct * ey_b

        x  = x_new + ex_w
        y  = y_new + ey_w
        th = (th_new + eth + np.pi) % (2*np.pi) - np.pi

        for i, pr in enumerate(self.p):
            pr.x = float(x[i]); pr.y = float(y[i]); pr.th = float(th[i])

    # --- Measurement model ---
    @staticmethod
    def _h_and_H(pose_xyth: tuple, mu_j: np.ndarray) -> tuple:
        """
        Range-bearing measurement model and Jacobian wrt landmark state.
        Args:
            pose_xyth: robot pose (x, y, theta)
            mu_j: landmark position (2-vector)
        Returns:
            zhat: predicted measurement (2-vector)
            H: Jacobian wrt landmark (2x2)
        """
        x, y, th = map(float, pose_xyth)
        dx = float(mu_j[0]) - x
        dy = float(mu_j[1]) - y
        q  = dx*dx + dy*dy + 1e-12
        r  = np.sqrt(q)
        zhat = np.array([r, wrap_angle(np.arctan2(dy, dx) - th)], dtype=np.float32)
        H = np.array([[dx/r,  dy/r],
                      [-dy/q,  dx/q]], dtype=np.float32)
        return zhat, H

    # --- Landmark initialization from first measurement ---
    @staticmethod
    def _init_from_meas(pose_xyth: tuple, z: np.ndarray, Q: np.ndarray) -> tuple:
        """
        Initialize landmark position and covariance from first range-bearing measurement.
        Args:
            pose_xyth: robot pose (x, y, theta)
            z: measurement (r, bearing)
            Q: measurement noise covariance (2x2)
        Returns:
            mu: landmark position (2-vector)
            Sigma: landmark covariance (2x2)
        """
        x, y, th = map(float, pose_xyth)
        r, b = map(float, z)
        mx = x + r * np.cos(th + b)
        my = y + r * np.sin(th + b)
        mu = np.array([mx, my], dtype=np.float32)

        _, H = FastSLAM._h_and_H((x, y, th), mu)
        # Landmark covariance via inverse (Hᵀ Q^{-1} H)^{-1} but avoid explicit inverse:
        # Solve H X = I  =>  X = H^{-1}; then Sigma = X Q Xᵀ
        I2 = np.eye(2, dtype=np.float32)
        try:
            X = np.linalg.solve(H, I2).astype(np.float32)
            Sigma = (X @ Q @ X.T).astype(np.float32)
        except np.linalg.LinAlgError:
            Sigma = np.diag([100.0, 100.0]).astype(np.float32)

        # Add tiny floor to keep Σ well-conditioned
        Sigma += np.eye(2, dtype=np.float32) * 1e-6
        return mu, Sigma

    # --- Correction / Update ---
    def update(self, z_list: list, id_list: list) -> None:
        """
        EKF update for each landmark in each particle + weight update.
        Args:
            z_list: list of measurements (each a 2-vector)
            id_list: list of landmark IDs (integers)
        Modifies:
            self.p: list of particles (in-place)
        """
        if not z_list:
            return

        Q = self.Q
        I2 = np.eye(2, dtype=np.float32)

        # Update each particle
        for pr in self.p:
            logw = 0.0
            pose = (pr.x, pr.y, pr.th)
            # Process each measurement
            for z, j in zip(z_list, id_list):
                if j not in pr.map:
                    mu, Sigma = self._init_from_meas(pose, z, Q)
                    pr.map[j] = [mu, Sigma, True]
                    continue

                mu, Sigma, _ = pr.map[j]

                # Predict measurement
                zhat, H = self._h_and_H(pose, mu)
                nu = np.array([z[0] - zhat[0], wrap_angle(z[1] - zhat[1])], dtype=np.float32)

                # Innovation cov S = H Σ Hᵀ + Q
                S = H @ Sigma @ H.T + Q

                # Kalman gain via linear solve (stable; no explicit inverse)
                # K = Σ Hᵀ S^{-1}
                HS = (H @ Sigma).astype(np.float32)  # (2,2)
                try:
                    KT = np.linalg.solve(S.T, HS)    # (2,2)
                # If S is singular / ill-conditioned, add tiny noise and retry
                except np.linalg.LinAlgError:
                    S = (S + np.eye(2, dtype=np.float32) * 1e-6).astype(np.float32)
                    KT = np.linalg.solve(S.T, HS)
                K = KT.T.astype(np.float32)          # (2,2)

                # Landmark EKF update
                mu    = (mu + K @ nu).astype(np.float32)
                Sigma = ((I2 - K @ H) @ Sigma).astype(np.float32)
                pr.map[j] = [mu, Sigma, True]

                # Particle weight update
                # log w += -0.5 * (nuᵀ S^{-1} nu + log|S| + 2 log(2π))
                try:
                    # Solve S y = nu
                    y = np.linalg.solve(S, nu.astype(np.float32))
                    m = float(nu.T @ y)
                    sign, logdet = np.linalg.slogdet(S.astype(np.float64))
                    if sign <= 0:
                        logdet = np.log(np.maximum(np.linalg.det(S.astype(np.float64)), 1e-9))
                    logw += -0.5 * (m + logdet + 2.0 * np.log(2.0 * np.pi))
                # If S is singular / ill-conditioned, skip weight update 
                except np.linalg.LinAlgError:
                    pass
            # Update particle weight
            pr.w = float(pr.w) * float(np.exp(logw))

        # Normalize weights
        W = np.array([pr.w for pr in self.p], dtype=np.float64)
        s = float(W.sum())
        if s <= 0.0 or not np.isfinite(s):
            W[:] = 1.0 / self.N
        else:
            W /= s
        for i, pr in enumerate(self.p):
            pr.w = float(W[i])

        # Resample if needed
        neff = 1.0 / (np.sum(W * W) + 1e-12)
        if neff < self.resample_neff_ratio * self.N:
            self._systematic_resample(W)


    # --- Resampling ---
    def _systematic_resample(self, W: np.ndarray) -> None:
        """
        Systematic resampling of particles according to weights W.
        Args:
            W: normalized weights (N-vector)
        Modifies:
            self.p: list of particles (in-place)
        """
        N = self.N
        # vectorized positions and index selection via searchsorted
        u0 = np.random.rand() / N
        positions = u0 + (np.arange(N, dtype=np.float64) / N)
        cdf = np.cumsum(W, dtype=np.float64)
        indexes = np.searchsorted(cdf, positions, side="left").astype(np.int32)

        invN = 1.0 / N
        new = []
        for k in indexes:
            src = self.p[int(k)]
            q = self.Particle(src.x, src.y, src.th, invN)
            # deep copy landmark states
            q.map = {lm_id: [mu.copy(), Sigma.copy(), seen]
                     for lm_id, (mu, Sigma, seen) in src.map.items()}
            new.append(q)
        self.p = new

    # --- Pose estimation ---
    def estimate_pose(self, top_k_frac: float = 1.0) -> np.ndarray:
        """
        Weighted average of top-k% particles (poses only).
        Args:
            top_k_frac: fraction of top particles to use (0,1]
        Returns:
            est: estimated pose (3-vector)
        """
        W = np.array([pr.w for pr in self.p], dtype=np.float64)
        idx = np.argsort(W)
        K = max(5, int(top_k_frac * len(self.p)))
        sel = idx[-K:]
        Wk = W[sel]
        Wk = Wk / (float(Wk.sum()) + 1e-12)
        xs = np.array([[self.p[i].x, self.p[i].y] for i in sel], dtype=np.float64)
        ths = np.array([self.p[i].th for i in sel], dtype=np.float64)
        ex, ey = np.sum(Wk[:, None] * xs, axis=0)
        c = np.sum(Wk * np.cos(ths))
        s = np.sum(Wk * np.sin(ths))
        eth = np.arctan2(s, c)
        return np.array([float(ex), float(ey), float(eth)], dtype=np.float32)

    # --- Utility ---
    def best_particle(self) -> Particle:
        return max(self.p, key=lambda t: t.w)

    def neff(self) -> float:
        W = np.array([pr.w for pr in self.p], dtype=np.float64)
        return float(1.0 / (np.sum(W * W) + 1e-12))


##### ☄ <span style="color:#a4d4a3">**Map & LiDAR Front-End** (grid-ID landmarks + quantized measurements)</span>

This cell provides a minimal <span style="color:#ffa500">**front-end**</span> for the FastSLAM core: a map with fast ray-casting and a simulated  <span style="color:#ffa500">**2D LiDAR**</span> that outputs range–bearing measurements  <span style="color:#ffa500">**with known correspondences**</span> (each hit is mapped to a persistent landmark ID defined by a grid cell).

<span style="color:#a4d4a3">**Map model** (Map2D):</span>

- Loads a floor-plan image and binarizes it: <span style="color:#00703c">**dark pixels = walls**</span>, the rest = free.

- Stores a boolean `wall_mask` of shape `(W,H)` (True for wall), and supports:

  - `nearest_free_to(xy)`: spawn helper returning the closest free pixel.

  - `cast_beam_fast(pos_xy, angle, max_r)`: ray-march along `angle` until the first wall hit or the boundary.

<span style="color:#a4d4a3">**Performance dials**</span>

- Fewer beams (`num_beams`), larger `ray_stride`, and coarser `grid_px` all speed up the front-end.

- Larger `range_step_px` / `bearing_step_deg` reduce landmarks and EKF load.

- If you need *even more* speed, subsample beams before passing to the SLAM back-end.

In [ ]:
import numpy as np
import pygame

# --- 2D map with fast ray-casting ---
class Map2D:
    """Loads a floor-plan image; black-ish pixels are walls; others free."""
    def __init__(self, panel_w: int, panel_h: int, ray_stride=2, floor_img_path='../figures/floor_plan.png'):
        self.panel_w = panel_w
        self.panel_h = panel_h
        self.ray_stride = int(ray_stride)

        # Try to load the map; fall back to blank free space if missing.
        try:
            surf = pygame.image.load(floor_img_path)
            surf = pygame.transform.smoothscale(surf, (panel_w, panel_h)).convert()
            self.surface = surf
            arr = pygame.surfarray.array3d(surf)  # (w, h, 3)
            self.wall_mask = np.all(arr < 128, axis=2).astype(np.bool_)  # True=wall
        except Exception:
            self.surface = pygame.Surface((panel_w, panel_h)).convert()
            self.surface.fill((240,240,240))
            self.wall_mask = np.zeros((panel_w, panel_h), dtype=np.bool_)

        self.free_mask = ~self.wall_mask
        free_xy = np.where(self.free_mask)
        self.free_coords = (np.stack(free_xy, axis=1).astype(np.int32)
                            if free_xy[0].size > 0 else np.zeros((0,2), dtype=np.int32))

    # --- Utility ---
    def nearest_free_to(self, target_xy: tuple) -> tuple:
        """
        Return the nearest free pixel to target_xy (or center if none).
        Args:
            target_xy: (x,y) pixel coordinates
        Returns:
            (x,y) of nearest free pixel (float)
        """
        tx, ty = int(target_xy[0]), int(target_xy[1])
        if 0<=tx<self.panel_w and 0<=ty<self.panel_h and (not self.wall_mask[tx, ty]):
            return float(tx), float(ty)
        if self.free_coords.shape[0] == 0:
            return float(self.panel_w//2), float(self.panel_h//2)
        # Vectorized nearest-neighbor over precomputed free pixels
        dif = self.free_coords - np.array([tx, ty], dtype=np.int32)[None, :]
        j = int(np.argmin(np.sum(dif.astype(np.int64)**2, axis=1)))
        x, y = self.free_coords[j]
        return float(x), float(y)

    def cast_beam_fast(self, pos_xy: tuple, angle: float, max_r: float) -> tuple:
        """
        Ray-march along angle until first wall hit or boundary.
        Args:
            pos_xy: (x,y) pixel coordinates of ray start
            angle: angle in radians
            max_r: max range in pixels  
        Returns:
            (hit_xy, distance_px): hit pixel (x,y) or None if no hit; distance in pixels
        """
        W, H = self.wall_mask.shape
        x0, y0 = float(pos_xy[0]), float(pos_xy[1])
        c, s = np.cos(angle), np.sin(angle)
        stride = max(1, int(self.ray_stride))

        # Vectorized march
        r = np.arange(0, int(max_r), stride, dtype=np.int32)             # (S,)
        X = (x0 + r * c).astype(np.int32)                                 # (S,)
        Y = (y0 + r * s).astype(np.int32)                                 # (S,)
        oob = (X < 0) | (X >= W) | (Y < 0) | (Y >= H)                     # (S,)
        hit_wall = np.zeros_like(oob, dtype=bool)
        inb = ~oob
        if np.any(inb):
            hit_wall[inb] = self.wall_mask[X[inb], Y[inb]]

        hit = oob | hit_wall
        if not np.any(hit):
            return None, int(max_r)

        k = int(np.argmax(hit))  # first hit index
        d = int(r[k])
        if oob[k]:
            return None, d
        else:
            return (int(X[k]), int(Y[k])), d

# --- 2D LiDAR with known correspondences ---
class Lidar:
    """
    Emits (r, bearing) with noise and **known correspondence**:
    - Each grid cell that a beam hits becomes a unique landmark ID.
    """
    def __init__(self, panel_w: int, panel_h: int, grid_px: int=20,
                 range_step_px: float=8.0, bearing_step_deg: float=1.0,
                 Q: np.ndarray = np.diag([4.0, np.deg2rad(5.0)**2]),
                 world_map: Map2D = None,
                 rng: np.random.Generator = None):
        """ 
        Args:
            panel_w, panel_h: map dimensions in pixels
            grid_px: size of grid cell for landmark IDs (pixels)
            range_step_px: quantization step for range (pixels)
            bearing_step_deg: quantization step for bearing (degrees)
            Q: measurement noise covariance (2x2)
            world_map: Map2D instance
            rng: numpy random number generator
        """
        self.panel_w = panel_w
        self.panel_h = panel_h
        self.grid_px = int(grid_px)
        self.range_step_px = float(range_step_px)
        self.bearing_step = np.deg2rad(float(bearing_step_deg))
        self.Q = Q.astype(np.float32)
        self.map = world_map
        self.rng = rng or np.random.default_rng(0)

        self.id_by_cell = {}  # (cx,cy) -> lm_id
        self.cell_by_id = []  # list index = lm_id -> (cx,cy)

    # --- Utility ---
    def cell_of_xy(self, x: float, y: float) -> tuple:
        """
        Return (cx,cy) grid cell coordinates for pixel (x,y).
        Args:
            x, y: pixel coordinates (float)
        Returns:
            (cx, cy): grid cell coordinates (int)
        """
        return (int(x)//self.grid_px, int(y)//self.grid_px)

    def cell_center_xy(self, cell: tuple) -> tuple:
        """
        Return (x,y) pixel coordinates of the center of the given grid cell.
        Args:
            cell: (cx, cy) grid cell coordinates (int)
        Returns:
            (x, y): pixel coordinates of cell center (float)
        """
        cx, cy = cell
        g = self.grid_px
        return (cx*g + 0.5*g, cy*g + 0.5*g)

    def _quantize_meas(self, r: float, phi: float) -> tuple:
        """
        Quantize range and bearing to nearest steps.
        Args:
            r: range (float)
            phi: bearing (float)
        Returns:
            (rq, pq): quantized range and bearing (float)
        """
        rq = self.range_step_px * round(r / self.range_step_px)
        pq = ((self.bearing_step * round(phi / self.bearing_step) + np.pi) % (2*np.pi)) - np.pi
        return rq, pq

    def _ensure_id(self, cell: tuple) -> int:
        """
        Ensure the given cell has a landmark ID; return it.
        Args:
            cell: (cx, cy) grid cell coordinates (int)
        Returns:
            lm_id: landmark ID (int)
        """
        if cell not in self.id_by_cell:
            lm_id = len(self.cell_by_id)
            self.id_by_cell[cell] = lm_id
            self.cell_by_id.append(cell)
        return self.id_by_cell[cell]

    def scan(self, gt_pose: tuple, heading: float, max_range_px: int=200,
             num_beams: int=180, fov_deg: int=360) -> tuple:
        """
        Simulate a LiDAR scan from the given pose and heading.
        Args:
            gt_pose: ground-truth robot pose (x, y, theta)
            heading: LiDAR heading relative to robot (radians)
            max_range_px: max range in pixels
            num_beams: number of beams to simulate
            fov_deg: field of view in degrees
        Returns:
            (z_all, c_all): list of measurements (each a 2-vector) and
                            corresponding landmark IDs (integers)
        """ 
        fov = np.deg2rad(fov_deg)
        angles = (heading + np.linspace(-fov/2, fov/2, num_beams, endpoint=False)) % (2*np.pi)

        # -------- Vectorized cast for all beams --------
        W, H = self.map.wall_mask.shape
        stride = max(1, int(self.map.ray_stride))
        S = int(max_range_px // stride) + 1
        r = (np.arange(S, dtype=np.int32) * stride)[None, :]          # (1,S)
        A = angles[:, None].astype(np.float32)                         # (B,1)

        x0, y0 = float(gt_pose[0]), float(gt_pose[1])
        X = (x0 + r * np.cos(A)).astype(np.int32)                      # (B,S)
        Y = (y0 + r * np.sin(A)).astype(np.int32)                      # (B,S)

        oob = (X < 0) | (X >= W) | (Y < 0) | (Y >= H)                  # (B,S)
        inb = ~oob
        wall = np.zeros_like(oob, dtype=bool)
        if np.any(inb):
            wall[inb] = self.map.wall_mask[X[inb], Y[inb]]
        hit = oob | wall
        has_hit = np.any(hit, axis=1)
        first_idx = np.argmax(hit, axis=1)                             # (B,)
        dists = np.where(has_hit, r.squeeze()[first_idx], float(max_range_px)).astype(np.float32)
        first_is_wall = np.zeros_like(has_hit, dtype=bool)
        if np.any(has_hit):
            first_is_wall[has_hit] = wall[np.arange(angles.size)[has_hit], first_idx[has_hit]]

        # keep only real wall hits
        keep = first_is_wall
        if not np.any(keep):
            return [], []

        # Hit pixels for kept beams
        rows = np.arange(angles.size, dtype=np.int32)[keep]
        cols = first_idx[keep].astype(np.int32)
        hx = X[rows, cols].astype(np.int32)
        hy = Y[rows, cols].astype(np.int32)

        # Cells and de-dup per scan (keep first occurrence)
        cx = (hx // self.grid_px).astype(np.int32)
        cy = (hy // self.grid_px).astype(np.int32)
        cells = np.stack([cx, cy], axis=1)                              # (K,2)
        # unique cells preserving first occurrence order
        if cells.shape[0] > 1:
            view = np.ascontiguousarray(cells).view(
                np.dtype((np.void, cells.dtype.itemsize * cells.shape[1]))
            )
            _, idx_first = np.unique(view, return_index=True)
            keep_idx = np.sort(idx_first)
        else:
            keep_idx = np.array([0], dtype=np.int32)

        beams_idx = rows[keep_idx]
        cells_sel = cells[keep_idx]
        d_sel = dists[keep][keep_idx]                                   # (K,)

        # Vectorized noisy measurements
        rng_std = np.sqrt(float(self.Q[0,0]))
        brg_std = np.sqrt(float(self.Q[1,1]))
        r_meas = d_sel + self.rng.normal(0.0, rng_std, size=d_sel.size).astype(np.float32)

        # true phi = world angle - robot heading
        phi_true = ((angles[beams_idx] - float(gt_pose[2]) + np.pi) % (2*np.pi)) - np.pi
        phi_meas = phi_true + self.rng.normal(0.0, brg_std, size=phi_true.size).astype(np.float32)

        # Quantize (vectorized)
        r_q = self.range_step_px * np.round(r_meas / self.range_step_px)
        p_step = self.bearing_step
        phi_q = ((p_step * np.round(phi_meas / p_step) + np.pi) % (2*np.pi)) - np.pi

        # Form outputs (lists as before)
        z_all = [np.array([float(rq), float(pq)], dtype=np.float32) for rq, pq in zip(r_q, phi_q)]
        c_all = []
        for (cx_i, cy_i) in cells_sel:
            j = self._ensure_id((int(cx_i), int(cy_i)))
            c_all.append(j)

        return z_all, c_all


##### 📲 <span style="color:#a4d4a3">**FastSLAM Pygame Demo**  (two-panel UI)</span>

This cell wires the <span style="color:#ffa500">**front-end**</span> (Map2D, Lidar) and the <span style="color:#ffa500">**FastSLAM core**</span> into an interactive `Pygame` demo with a <span style="color:#ffa500">**two-panel layout**</span>:

- <span style="color:#00703c">**Left panel:**</span> ground-truth robot on the floor-plan (image), plus a rotating visualization beam.

- <span style="color:#00703c">**Right panel** (static world frame):</span> estimated robot pose, particle cloud, best-particle landmarks (with optional covariance ellipses), and estimated trajectory.

The default map path is `../figures/floor_plan.png` (falls back to a blank free map if not found), but you can also change it with an occupancy map that you built in the first module.

<span style="color:#a4d4a3">**Controls**</span>

- <span style="color:#ffa500">**,**</span> / <span style="color:#ffa500">**.**</span> decrease/increase <span style="color:#ffa500">**beam subsampling**</span> (trade accuracy vs. speed)

- <span style="color:#ffa500">**N**</span> toggle <span style="color:#ffa500">**nearest-K**</span> thinning of measurements (limits EKF load)

- <span style="color:#ffa500">**P**</span> toggle particle rendering

- <span style="color:#ffa500">**C**</span> toggle covariance ellipses

- <span style="color:#ffa500">**G**</span> re-initialize particles (around current map center)

- <span style="color:#ffa500">**K**</span> reset <span style="color:#ffa500">**GT**</span> and <span style="color:#ffa500">**EST**</span> pose to map center; clear trajectories

<span style="color:#a4d4a3">**Performance tips**</span>

- For smoother rendering: reduce `num_beams`, increase `beam_subsample_step` or `ray_stride`, reduce `Np`, increase `grid_px`, or press <span style="color:#ffa500">**C**</span> to disable covariance ellipses.


In [11]:
#--- Main application with Pygame ---
import sys
import numpy as np
import pygame
from collections import deque

# --- Small helper ---
def i2(p):
    """(x,y) -> (int,int)"""
    return (int(p[0]), int(p[1]))

# --- Config ---
class Config:
    # Window/panels
    panel_w, panel_h = 500, 400
    window_size = (panel_w*2, panel_h)

    # Scan/visual
    num_beams = 180
    scan_ms   = 180
    fov_deg   = 360
    ray_stride = 2
    traj_max   = 200

    # Discretization (one landmark id per grid cell hit by LiDAR)
    grid_px = 16
    range_step_px = 4.0
    bearing_step_deg = 4.0

    # Motion (keyboard odometry: [dr1, dt, dr2])
    trans_step = 12.0
    rot_step   = np.deg2rad(12.0)

    # Sensor noise (pixels & radians)
    Q = np.diag([2.0**2, (2.0*np.pi/180.0)**2]).astype(np.float32)
    R_x = np.diag([1.0**2, 1.0**2, (2.0*np.pi/180.0)**2]).astype(np.float32)
    
    # FastSLAM params
    Np = 75   # number of particles
    resample_neff_ratio = 0.5   # resample when N_eff < ratio * N

    # Landmark thinning
    use_nearest = True
    nearest_k_seen = 25

    # Beam sub-sampling
    beam_subsample_step = 2

    # Drawing perf
    particle_draw_stride = 2

    # Start seed
    seed = 0

#--- Simple panel wrapper ---
class Panel:
    def __init__(self, screen, rect): 
        self.screen, self.rect = screen, rect
    @property
    def ox(self): return self.rect[0]
    def clear(self, color):
        pygame.draw.rect(self.screen, color, self.rect)
        pygame.draw.rect(self.screen, (200,200,200), self.rect, 2)
    def blit(self, surf, at=(0,0)):
        x,y,_,_ = self.rect; self.screen.blit(surf, (x+at[0], y+at[1]))
    def polygon(self, color, pts):
        x,y,_,_ = self.rect; P = [(x+int(px), y+int(py)) for (px,py) in pts]
        pygame.draw.polygon(self.screen, color, P)
    def circle(self, color, p, r, width=0):
        """width=0 => filled; width>0 => outline."""
        x,y,_,_ = self.rect
        pygame.draw.circle(self.screen, color, (x+int(p[0]), y+int(p[1])), int(r), width)
    def line(self, color, p0, p1, w=1):
        x,y,_,_ = self.rect; pygame.draw.line(self.screen, color, (x+int(p0[0]), y+int(p0[1])), (x+int(p1[0]), y+int(p1[1])), w)
    def text(self, font, s, color, pos):
        x,y,_,_ = self.rect; self.screen.blit(font.render(s, True, color), (x+pos[0], y+pos[1]))

# --- Main application ---
class App:
    def __init__(self, cfg: Config, floor_img_path='../figures/floor_plan.png'):
        pygame.init()
        self.cfg = cfg
        self.screen = pygame.display.set_mode(cfg.window_size)
        pygame.display.set_caption("GT (left) | FastSLAM (right, static map)")
        self.clock = pygame.time.Clock()
        self.font  = pygame.font.SysFont(None, 18)
        self.rng   = np.random.default_rng(cfg.seed)

        # Panels
        self.left  = Panel(self.screen, (0, 0, cfg.panel_w, cfg.panel_h))
        self.right = Panel(self.screen, (cfg.panel_w, 0, cfg.panel_w, cfg.panel_h))

        # Map + lidar
        self.map = Map2D(cfg.panel_w, cfg.panel_h, ray_stride=cfg.ray_stride, floor_img_path=floor_img_path)
        self.lidar = Lidar(cfg.panel_w, cfg.panel_h, grid_px=cfg.grid_px,
                           range_step_px=cfg.range_step_px,
                           bearing_step_deg=cfg.bearing_step_deg,
                           Q=cfg.Q, world_map=self.map, rng=self.rng)

        # GT start at CENTER (nearest free)
        cx, cy = cfg.panel_w//2, cfg.panel_h//2
        gx, gy = self.map.nearest_free_to((cx, cy))
        gth    = 0.0
        self.gt_pose = np.array([gx, gy, gth], dtype=np.float32)
        self.gt_traj = []

        # FastSLAM core (independent)
        self.fs  = FastSLAM(N=cfg.Np, Q=cfg.Q, R_x=cfg.R_x, rng=self.rng,
                            resample_neff_ratio=cfg.resample_neff_ratio)
        self.fs.global_initialize(gx, gy, th_sd_deg=10.0, pos_sd=3.0)

        self.est_pose = np.array([gx, gy, 0.0], dtype=np.float32)
        self.est_traj = []

        # Scan timer
        self.SCAN_EVENT = pygame.USEREVENT + 1
        pygame.time.set_timer(self.SCAN_EVENT, cfg.scan_ms)
        self.beam_angle = 0.0
        self.beam_speed = 2*np.pi*10
        self.max_range_px = 300

        # UI toggles
        self.draw_particles = True
        self.draw_cov = True

    # --- Motion model (shared helper) ---
    @staticmethod
    def motion_model_odometry(u, x_prev: np.ndarray) -> np.ndarray:
        """Apply odometry u = [dr1, dt, dr2] to previous pose x_prev = [x,y,th].
        Args:
            u: odometry 3-vector (dr1, dt, dr2)
            x_prev: previous pose (3-vector)
        Returns:
            x_new: new pose (3-vector)
        """
        dr1, dt, dr2 = u
        x_new = x_prev[0] + dt*np.cos(x_prev[2] + dr1)
        y_new = x_prev[1] + dt*np.sin(x_prev[2] + dr1)
        th_new= wrap_angle(x_prev[2] + dr1 + dr2)
        return np.array([x_new, y_new, th_new], dtype=np.float32)

    # --- Step motion + FastSLAM predict ---
    def step_motion(self, cmd: np.ndarray):
        """Apply odometry command to GT and FastSLAM.
        Args:
            cmd: odometry command (3-vector)
        """
        self.gt_pose = self.motion_model_odometry(cmd, self.gt_pose)
        self.gt_traj.append(self.gt_pose.copy())
        self.gt_traj = self.gt_traj[-self.cfg.traj_max:]

        self.fs.predict(cmd)
        self.est_pose = self.fs.estimate_pose(top_k_frac=1.0)
        self.est_traj.append(self.est_pose.copy())
        self.est_traj = self.est_traj[-self.cfg.traj_max:]

    # --- Scan + FastSLAM update ---
    def step_scan(self):
        """Simulate a LiDAR scan from GT pose; FastSLAM update."""
        heading = float(self.gt_pose[2])
        z_all, c_all = self.lidar.scan(self.gt_pose, heading,
                                      max_range_px=self.max_range_px,
                                      num_beams=self.cfg.num_beams,
                                      fov_deg=self.cfg.fov_deg)
        if not z_all:
            return

        # Nearest-K thinning (optional)
        if self.cfg.use_nearest and (self.est_pose is not None):
            cx, cy = float(self.est_pose[0]), float(self.est_pose[1])
            ids = np.array(list(set(int(j) for j in c_all)), dtype=np.int32)
            centers = np.array([self.lidar.cell_center_xy(self.lidar.cell_by_id[j]) for j in ids],
                               dtype=np.float32)
            d = np.linalg.norm(centers - np.array([cx,cy], dtype=np.float32)[None,:], axis=1)
            k = min(self.cfg.nearest_k_seen, ids.size)
            keep = np.argpartition(d, k-1)[:k]
            keep_ids = set(int(i) for i in ids[keep])
            batch = [(z,j) for (z,j) in zip(z_all, c_all) if (j in keep_ids)]
            if batch:
                z_all, c_all = map(list, zip(*batch))

        # Subsample beams
        if self.cfg.beam_subsample_step > 1:
            z_all = z_all[::self.cfg.beam_subsample_step]
            c_all = c_all[::self.cfg.beam_subsample_step]

        # RBPF correct
        self.fs.update(z_all, c_all)
        self.est_pose = self.fs.estimate_pose(top_k_frac=1.0)

    # --- Drawing ---
    def draw_left(self):
        self.left.clear((30,30,30))
        self.left.blit(self.map.surface, (0,0))

        # GT traj
        for i in range(1, len(self.gt_traj)):
            p0 = i2(self.gt_traj[i-1][:2]); p1 = i2(self.gt_traj[i][:2])
            pygame.draw.line(self.screen, (0,150,255), (self.left.ox+p0[0], p0[1]),
                             (self.left.ox+p1[0], p1[1]), 2)

        # GT robot (triangle)
        gx,gy,gth = map(float, self.gt_pose)
        pts = [
            (gx + 15*np.cos(gth),     gy + 15*np.sin(gth)),
            (gx + 10*np.cos(gth+2.5), gy + 10*np.sin(gth+2.5)),
            (gx + 10*np.cos(gth-2.5), gy + 10*np.sin(gth-2.5))
        ]
        self.left.polygon((0,150,255), pts)

        # rotating beam visualization
        bx,by = i2(self.gt_pose[:2])
        hit, dist = self.map.cast_beam_fast(self.gt_pose[:2], self.beam_angle, self.max_range_px)
        ex,ey = (hit if hit else (int(bx + self.max_range_px*np.cos(self.beam_angle)),
                                 int(by + self.max_range_px*np.sin(self.beam_angle))))
        pygame.draw.line(self.screen, (255,0,0), (self.left.ox+bx, by), (self.left.ox+ex, ey), 2)

    def draw_right(self):
        self.right.clear((255,255,255))

        # Estimated trajectory
        if len(self.est_traj) >= 2:
            for i in range(1, len(self.est_traj)):
                p0 = self.est_traj[i-1][:2]; p1 = self.est_traj[i][:2]
                self.right.line((120,200,160), p0, p1, 2)

        # Particles
        if self.draw_particles:
            stride = max(1, self.cfg.particle_draw_stride)
            for idx, pr in enumerate(self.fs.p):
                if (idx % stride) == 0:
                    self.right.circle((0,180,120), (pr.x, pr.y), 1)

        # Estimated robot
        if self.est_pose is not None:
            px,py,pth = map(float, self.est_pose)
            self.right.circle((0,180,120), (px,py), 6, width=2)  # hollow outline
            hx = px + 14*np.cos(pth); hy = py + 14*np.sin(pth)
            self.right.line((0,180,120), (px,py), (hx,hy), 2)

        # Landmarks from best particle
        best = self.fs.best_particle()
        for lm_id, (mu, Sigma, seen) in best.map.items():
            if not seen: continue
            self.right.circle((255,0,0), (float(mu[0]), float(mu[1])), 2)
            if self.draw_cov:
                self.draw_cov_ellipse_world(mu, Sigma, (120,60,60))

        # HUD
        fps = self.clock.get_fps(); neff = self.fs.neff(); bstep = self.cfg.beam_subsample_step
        hud1 = (f"FPS:{fps:4.1f} | N:{self.cfg.Np} | Neff:{neff:5.1f} "
                f"| Beams:{self.cfg.num_beams}/{bstep} | Grid:{self.cfg.grid_px}px "
                f"| LMs:{len(self.lidar.cell_by_id)}")
        self.right.text(self.font, hud1, (10,10,10), (10,0))

    def draw_cov_ellipse_world(self, mu, Sigma, color, nsig=2.0):
        # Guard against NaN/inf or absurd positions
        if not np.all(np.isfinite(mu)) or not np.all(np.isfinite(Sigma)):
            return
        if abs(mu[0]) > 1e6 or abs(mu[1]) > 1e6:
            return

        try:
            vals, vecs = np.linalg.eigh(Sigma)
        except np.linalg.LinAlgError:
            return
        vals = np.maximum(vals, 1e-12)
        order = np.argsort(vals)[::-1]
        vals = vals[order]; vecs = vecs[:, order]
        w = 2*nsig*np.sqrt(vals[0]); h = 2*nsig*np.sqrt(vals[1])
        ang = np.degrees(np.arctan2(vecs[1,0], vecs[0,0]))

        rect = pygame.Rect(0, 0, max(1, int(w)), max(1, int(h)))
        try:
            rect.center = (int(mu[0]), int(mu[1]))
        except Exception:
            return  # safety, in case mu still misbehaves

        el = pygame.Surface(rect.size, pygame.SRCALPHA)
        pygame.draw.ellipse(el, color, el.get_rect(), 2)
        rot = pygame.transform.rotate(el, -ang)
        rs = rot.get_rect(center=(self.right.ox + rect.centerx, rect.centery))
        self.screen.blit(rot, rs)

    # --- Main loop ---
    def run(self):
        running = True
        while running:
            dt = self.clock.tick(30) / 1000.0
            self.beam_angle = (self.beam_angle + self.beam_speed*dt) % (2*np.pi)
            for ev in pygame.event.get():
                if ev.type == pygame.QUIT:
                    running = False
                elif ev.type == pygame.KEYDOWN:
                    if   ev.key == pygame.K_UP:    cmd = np.array([0.0, self.cfg.trans_step, 0.0], dtype=np.float32)
                    elif ev.key == pygame.K_LEFT:  cmd = np.array([-self.cfg.rot_step, 0.0, 0.0], dtype=np.float32)
                    elif ev.key == pygame.K_RIGHT: cmd = np.array([ self.cfg.rot_step, 0.0, 0.0], dtype=np.float32)
                    elif ev.key == pygame.K_COMMA: self.cfg.beam_subsample_step = max(1, self.cfg.beam_subsample_step-1); cmd=None
                    elif ev.key == pygame.K_PERIOD:self.cfg.beam_subsample_step = min(12, self.cfg.beam_subsample_step+1); cmd=None
                    elif ev.key == pygame.K_n:     self.cfg.use_nearest = not self.cfg.use_nearest; cmd=None
                    elif ev.key == pygame.K_p:     self.draw_particles = not self.draw_particles; cmd=None
                    elif ev.key == pygame.K_c:     self.draw_cov = not self.draw_cov; cmd=None
                    elif ev.key == pygame.K_g:     # re-init particles around center
                        cx, cy = self.cfg.panel_w//2, self.cfg.panel_h//2
                        gx, gy = self.map.nearest_free_to((cx, cy))
                        self.fs.global_initialize(gx, gy, th_sd_deg=10.0, pos_sd=3.0); cmd=None
                    elif ev.key == pygame.K_k:
                        cx, cy = self.cfg.panel_w//2, self.cfg.panel_h//2
                        gx, gy = self.map.nearest_free_to((cx, cy))
                        self.gt_pose  = np.array([gx, gy, 0.0], dtype=np.float32)
                        self.est_pose = self.gt_pose.copy()
                        self.gt_traj.clear(); self.est_traj.clear(); cmd=None
                    else:
                        cmd=None
                    if cmd is not None:
                        self.step_motion(cmd)
                elif ev.type == self.SCAN_EVENT:
                    self.step_scan()

            self.draw_left(); self.draw_right(); pygame.display.flip()

        pygame.quit(); sys.exit()

# run_demo.py
if __name__ == "__main__":
    cfg = Config()
    App(cfg).run()

SystemExit: 

---

#### ⏱️ <span style="color:#a4d4a3">**FastSLAM Complexity**</span>

- <span style="color:#ffa500">**Update robot particles**</span> based on the control $O(N)$

- <span style="color:#ffa500">**Incorporate an obseravtion**</span> into the Kalman Filters $O(N)$

- <span style="color:#ffa500">**Resample particle**</span> set $O(N \cdot M)$

where:

- $N$ is the number of particles
- $M$ is the number of landmarks

Thus $O(N \cdot M)$ for FastSLAM1.0

#### ⬆️ <span style="color:#a4d4a3">**From FastSLAM1.0 to FastSLAM2.0**</span>

<span style="color:#00703c">**FastSLAM1.0**</span> uses the motion model as the proposal distribution:

$$
x_t^{[k]} \sim p(x_t \mid x_{t-1}^{[k]}, u_t)
$$

*But, is there <span style="color:#ffa500">**a better distribution**</span> to sample from?*

- <span style="color:#00703c">**FastSLAM 2.0**</span> considers also the measurements during sampling.

- Especially useful if an <span style="color:#ffa500">**accurate sensor is used**</span> (e.g. predictions from scan matching, compared to the motion noise).

- <span style="color:#00703c">**FastSLAM 2.0**</span>  samples from:

    $$
    x_t^{[k]} \sim p(x_t \mid x_{t-1}^{[k]}, u_{1:t}. z_{1:t})
    $$

- Results in a <span style="color:#ffa500">**more peaked proposal distribution**</span>.

- <span style="color:#ffa500">**Less particles**</span> required.

- More robust and accurate but also <span style="color:#ffa500">**more complex**</span>.


---

#### ✔️ <span style="color:#a4d4a3">Summary</span>

- Uses a <span style="color:#ffa500">**Particle Filter**</span> to model the belief over <span style="color:#ffa500">**robot paths**</span>.

- <span style="color:#ffa500">**Factorizes**</span> the SLAM posterior into <span style="color:#ffa500">**low-dimensional**</span> problems.

- <span style="color:#ffa500">**Samples only the robot’s path**</span>; for each particle, <span style="color:#ffa500">**computes landmarks**</span> (independent EKFs) <span style="color:#ffa500">**given the path**</span>.

- Particle <span style="color:#ffa500">**weights**</span> measure how <span style="color:#ffa500">**consistent**</span> each sample’s world is with <span style="color:#ffa500">**what the system actually perceives**</span>.

- Per-particle data association.

- No robot pose uncertainty in the per-particle data association.

- <span style="color:#00703c">**FastSLAM1.0**</span> and <span style="color:#00703c">**FastSLAM2.0**</span> differ in the <span style="color:#ffa500">**proposal distribution**</span>.

- Complexity $O(N \cdot \log{M})$.

- <span style="color:#ffa500">**Scales well**</span> (1 million+ features).

- <span style="color:#ffa500">**Robust to ambiguities**</span> in the data association.

- <span style="color:#ffa500">**Advantages compared to the classical EKF**</span> approach (especially with non-liberalities).


---

### 📚 <span style="color:#a4d4a3">**Reading Material**</span>

**FastSLAM**
- Thrun et al.: *"Probabilistic Robotics"*, **Chapter 13.1-13.3, 13.8**
 
- Montemerlo, Thrun, Kollar amd Wegbreit: ***"FastSLAM: A Factored Solution to the Simultaneous Localization and Mapping Problem"***, 2002

- Montemerlo and Thrun: ***"Simultaneous Localization and Mapping with Uknown Data Association Using FastSLAM"***, 2003